List and Create S3 Buckets 

In [ ]:
import boto3

s3 = boto3.client("s3")

In [ ]:
response = s3.list_buckets()

In [ ]:
def create_bucket(bucket_name):
    s3.create_bucket(Bucket=bucket_name)
    print("S3 bucket created!")

bucket_name = "kev-ml-test-bucket"
create_bucket(bucket_name)

Upload a File to S3

In [ ]:
import os

def upload(file_path, object_name=None):
    if object_name is None or object_name == "":
        object_name = os.path.basename(file_path)
    s3.upload_file(file_path, bucket_name, object_name)

upload("data/index.html", "index.html")

List Objects in S3 Bucket

In [ ]:
def list_objects():
    response = s3.list_objects_v2(Bucket=bucket_name)
    for obj in response["Contents"]:
        print(obj)

list_objects()

Download S3 File to Local

In [ ]:
def download_file(object_name, file_path):
    if not os.path.exists(os.path.dirname(file_path)):
        os.makedirs(os.path.dirname(file_path))
    s3.download_file(bucket_name, object_name, file_path)

download_file("index.html", "downloads/index.html")

Upload Directory to S3

In [ ]:
def upload_directory(dir_path, s3_prefix):
    for root, dirs, files in os.walk("data"):
        for file in files:
            file_path = os.path.join(root, file).replace("\\", "/")
            relpath = os.path.relpath(file_path, dir_path)
            s3_key = os.path.join(s3_prefix, relpath).replace("\\", "/")
            s3.upload_file(file_path, bucket_name, s3_key)

upload_directory("data", "s3_data")

Download S3 Directory to Local

In [ ]:
local_path = "s3_downloads"
s3_prefix = "s3_data"

def download_dir(local_path, s3_prefix):
    os.makedirs(local_path, exist_ok=True)
    paginator = s3.get_paginator("list_objects_v2")
    
    for result in paginator.paginate(Bucket=bucket_name, Prefix=s3_prefix):
        if "Contents" in result.keys():
            for key in result["Contents"]:
                s3_key = key["Key"]
                local_file = os.path.join(local_path, os.path.relpath(s3_key, s3_prefix))
                s3.download_file(bucket_name, s3_key, local_file)

download_dir(local_path, s3_prefix)

Delete All Files in S3 Bucket

In [ ]:
def delete_objects():
    response = s3.list_objects_v2(Bucket=bucket_name)
    if "Contents" in response.keys():
        for obj in response["Contents"]:
            s3.delete_object(Bucket=bucket_name, Key=obj["Key"])
        print("Files deleted")

delete_objects()

Delete S3 Bucket

In [ ]:
s3.delete_bucket(Bucket=bucket_name)